# NetworkX - beagle Clustering
------------------

- Try to get KarateClub working.
- Getting correlation coefficitions / centrality measures for nodes in ancestor chain

# Preparation

In [1]:
import pandas as pd, numpy as np, networkx as nx
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import chi2
from sklearn.linear_model.logistic import LogisticRegression
pd.options.display.max_columns = 500
pd.options.display.max_colwidth = 500

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.linear_model.logistic module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
df_zeek = pd.read_json("combined_zeek.log", lines = True)
df_zeek.replace('nan', np.nan, inplace=True)
print(type(df_zeek))
print(df_zeek.shape)
#print(df_zeek.EventID.value_counts())
df_zeek.head()

<class 'pandas.core.frame.DataFrame'>
(2140, 148)


,@stream,@system,@proc,ts,uid,id_orig_h,id_orig_p,id_resp_h,id_resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,orig_l2_addr,resp_l2_addr,rtt,named_pipe,endpoint,operation,trans_id,query,qclass,qclass_name,qtype,qtype_name,rcode,rcode_name,AA,TC,RD,RA,Z,answers,TTLs,rejected,analyzer,failure_reason,fuid,tx_hosts,rx_hosts,conn_uids,source,depth,analyzers,mime_type,filename,is_orig,seen_bytes,total_bytes,missing_bytes,overflow_bytes,timedout,md5,sha1,request_type,client,success,error_msg,till,forwardable,renewable,cipher,note,msg,sub,src,dst,p,actions,suppress_for,action,path,name,size,times_modified,times_accessed,times_created,times_changed,share_type,version,curve,server_name,resumed,established,cert_chain_fuids,client_cert_chain_fuids,subject,issuer,validation_status,resp_certificate_sha1,ja3,ja3s,notice,peer,addl,id,certificate_version,certificate_serial,certificate_subject,certificate_issuer,certificate_not_valid_before,certificate_not_valid_after,certificate_key_alg,certificate_sig_alg,certificate_key_type,certificate_key_length,certificate_exponent,basic_constraints_ca,extracted,extracted_cutoff,trans_depth,method,host,uri,user_agent,request_body_len,response_body_len,status_code,status_msg,tags,resp_fuids,resp_mime_types,client_header_names,orig_fuids,orig_mime_types,machine,compile_ts,os,subsystem,is_exe,is_64bit,uses_aslr,uses_dep,uses_code_integrity,uses_seh,has_import_table,has_export_table,has_cert_table,has_debug_data,section_names
0,conn,bobs.bigwheel.local,zeek,1.588205e+09,Cvf4XX17hSAgXDdGEd,10.0.1.6,54243.0,10.0.0.4,53.0,udp,dns,0.001528,39.0,55.0,SF,0.0,Dd,1.0,67.0,1.0,83.0,00:0d:3a:13:e3:69,12:34:56:78:9a:bc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,conn,bobs.bigwheel.local,zeek,1.588205e+09,CJ21Le4zsTUcyKKi98,10.0.1.6,56880.0,10.0.0.4,445.0,tcp,"gssapi,smb,krb",10.761077,4020.0,1681.0,RSTO,0.0,ShADdaR,12.0,1307.0,11.0,2133.0,00:0d:3a:13:e3:69,12:34:56:78:9a:bc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,conn,bobs.bigwheel.local,zeek,1.588205e+09,CnOP7t1eGGHf6LFfuk,10.0.1.6,65108.0,10.0.0.4,53.0,udp,dns,0.001599,36.0,104.0,SF,0.0,Dd,1.0,64.0,1.0,132.0,00:0d:3a:13:e3:69,12:34:56:78:9a:bc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,conn,bobs.bigwheel.local,zeek,1.588205e+09,CvxbPE3MuO7boUdSc8,10.0.1.6,138.0,10.0.1.255,138.0,udp,NaN,NaN,NaN,NaN,S0,0.0,D,1.0,229.0,0.0,0.0,00:0d:3a:13:e3:69,ff:ff:ff:ff:ff:ff,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [3]:
df_label = pd.read_json('./df_labelled_coalesce.json', lines=True)
df_label['TP_Binary'] = df_label.TP_Label!='0'
df_label.replace('-', np.nan, inplace=True)
print(type(df_label))
print(df_label.shape)
print(df_label.EventID.value_counts())
df_label.head(1)

<class 'pandas.core.frame.DataFrame'>
(198760, 390)
12       61158
10       39286
7        20259
13       17542
4658     10973
         ...  
172          1
46           1
50103        1
306          1
41           1
Name: EventID, Length: 165, dtype: int64


,@timestamp,@version,Category,Channel,EventID,EventReceivedTime,EventTime,EventType,ExecutionProcessID,Hostname,IpAddress,IpPort,Keywords,LogonGuid,Message,Opcode,OpcodeValue,ProviderGuid,RecordNumber,ServiceName,ServiceSid,Severity,SeverityValue,SourceModuleName,SourceModuleType,SourceName,Status,TargetDomainName,TargetUserName,Task,ThreadID,TicketEncryptionType,TicketOptions,TransmittedServices,Version,host,port,tags,TP_Label,Action,ActivityID,CalloutKey,CalloutName,ChangeType,Conditions,FilterId,FilterKey,FilterName,FilterType,LayerId,LayerKey,LayerName,ProcessId,ProviderKey,ProviderName,UserName,UserSid,Weight,AccessList,AccessMask,AccessReason,HandleId,ObjectName,ObjectServer,ObjectType,PrivilegeList,ProcessName,ResourceAttributes,RestrictedSidCount,SubjectDomainName,SubjectLogonId,SubjectUserName,SubjectUserSid,TransactionId,ShareLocalPath,ShareName,RelativeTargetName,CommandLine,MandatoryLabel,NewProcessId,NewProcessName,ParentProcessName,TargetLogonId,TargetUserSid,TokenElevationType,Service,DisabledPrivilegeList,EnabledPrivilegeList,CallerProcessId,CallerProcessName,TargetSid,AuthenticationPackageName,ElevatedToken,ImpersonationLevel,KeyLength,LmPackageName,LogonProcessName,LogonType,RestrictedAdminMode,TargetLinkedLogonId,TargetOutboundDomainName,TargetOutboundUserName,VirtualAccount,WorkstationName,SourceHandleId,SourceProcessId,TargetHandleId,TargetProcessId,AccountName,AccountType,ContextInfo,Domain,Payload,UserID,NewThreadId,RuleName,SourceImage,SourceProcessGuid,StartAddress,StartFunction,StartModule,TargetImage,TargetProcessGuid,UtcTime,MessageNumber,MessageTotal,Path,ScriptBlockId,ScriptBlockText,AlgorithmName,KeyName,KeyType,Operation,ReturnCode,Archived,Hashes,Image,IsExecutable,ProcessGuid,TargetFilename,User,CreationUtcTime,Company,Description,FileVersion,ImageLoaded,OriginalFileName,Product,Signature,SignatureStatus,Signed,DestinationHostname,DestinationIp,DestinationIsIpv6,DestinationPort,DestinationPortName,Initiated,Protocol,SourceHostname,SourceIp,SourceIsIpv6,SourcePort,SourcePortName,NewSd,OldSd,PipeName,CurrentDirectory,IntegrityLevel,LogonId,ParentCommandLine,ParentImage,ParentProcessGuid,ParentProcessId,TerminalSessionId,CallTrace,GrantedAccess,SourceProcessGUID,SourceThreadId,TargetProcessGUID,Device,TargetObject,Details,Application,FilterRTID,LayerRTID,SourceAddress,DestAddress,DestPort,Direction,RemoteMachineID,RemoteUserID,BootMenuPolicy,NewValue,NewValueType,ObjectValueName,OldValue,OldValueType,OperationType,ProfileUsed,RuleId,QueryName,QueryResults,QueryStatus,PreviousCreationUtcTime,EventCountTotal,EventIdx,GroupMembership,param1,Properties,Hash,PreAuthType,ClientProcessId,CountOfCredentialsReturned,ProcessCreationTime,ReadOperation,TargetName,Type,CalloutId,CalloutType,CorruptionActionState,DeviceName,DriveName,Config,IsTestConfig,DirtyPages,HiveName,HiveNameLength,KeysUpdated,PackageName,Workstation,DeviceNameLength,DeviceTime,DeviceVersionMajor,DeviceVersionMinor,FinalStatus,UserData,ServiceAccount,ServiceFileName,ServiceStartType,ServiceType,param10,param11,param2,param3,param4,param5,param6,param7,param8,param9,SubLayerKey,SubLayerName,SubLayerType,Default SD String:,ClassId,ClassName,CompatibleIds,DeviceDescription,DeviceId,LocationInformation,VendorIds,TargetProcessName,SecurityPackageName,AdditionalInfo,AdditionalInfo2,ProviderType,RuleAttr,AdapterName,AdapterSuffixName,DnsServerList,ErrorCode,Ipaddress,Sent UpdateServer,TaskContentNew,TaskName,ActiveProfile,MiniportName,MiniportNameLen,GroupPolicyApplied,LogDroppedPacketsEnabled,LogSuccessfulConnectionsEnabled,MulticastFlowsEnabled,OperationMode,Profile,RemoteAdminEnabled,TargetInfo,TargetLogonGuid,TargetServerName,ClientCreationTime,TimeSource,ImagePath,StartType,ProfileChanged,BootType,LoadOptions,EntryCount,KeyFilePath,ProviderContextKey,ProviderContextName,ProviderContextType,DwordVal,InterfaceGuid,InterfaceName,NewProfile,OldProfile,InstanceId,MaxRunspaces,MinRunspaces,CurrentStratumNumber,TimeSourceRefId,ReasonForRejection,

In [4]:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html
#    In [9]: result = pd.concat([df1, df4], axis=1, sort=False)
df_label = df_label.append(df_zeek, ignore_index=True, sort=False)

In [5]:
df_label.shape

(200900, 536)

In [6]:
df_label.head(1)

,@timestamp,@version,Category,Channel,EventID,EventReceivedTime,EventTime,EventType,ExecutionProcessID,Hostname,IpAddress,IpPort,Keywords,LogonGuid,Message,Opcode,OpcodeValue,ProviderGuid,RecordNumber,ServiceName,ServiceSid,Severity,SeverityValue,SourceModuleName,SourceModuleType,SourceName,Status,TargetDomainName,TargetUserName,Task,ThreadID,TicketEncryptionType,TicketOptions,TransmittedServices,Version,host,port,tags,TP_Label,Action,ActivityID,CalloutKey,CalloutName,ChangeType,Conditions,FilterId,FilterKey,FilterName,FilterType,LayerId,LayerKey,LayerName,ProcessId,ProviderKey,ProviderName,UserName,UserSid,Weight,AccessList,AccessMask,AccessReason,HandleId,ObjectName,ObjectServer,ObjectType,PrivilegeList,ProcessName,ResourceAttributes,RestrictedSidCount,SubjectDomainName,SubjectLogonId,SubjectUserName,SubjectUserSid,TransactionId,ShareLocalPath,ShareName,RelativeTargetName,CommandLine,MandatoryLabel,NewProcessId,NewProcessName,ParentProcessName,TargetLogonId,TargetUserSid,TokenElevationType,Service,DisabledPrivilegeList,EnabledPrivilegeList,CallerProcessId,CallerProcessName,TargetSid,AuthenticationPackageName,ElevatedToken,ImpersonationLevel,KeyLength,LmPackageName,LogonProcessName,LogonType,RestrictedAdminMode,TargetLinkedLogonId,TargetOutboundDomainName,TargetOutboundUserName,VirtualAccount,WorkstationName,SourceHandleId,SourceProcessId,TargetHandleId,TargetProcessId,AccountName,AccountType,ContextInfo,Domain,Payload,UserID,NewThreadId,RuleName,SourceImage,SourceProcessGuid,StartAddress,StartFunction,StartModule,TargetImage,TargetProcessGuid,UtcTime,MessageNumber,MessageTotal,Path,ScriptBlockId,ScriptBlockText,AlgorithmName,KeyName,KeyType,Operation,ReturnCode,Archived,Hashes,Image,IsExecutable,ProcessGuid,TargetFilename,User,CreationUtcTime,Company,Description,FileVersion,ImageLoaded,OriginalFileName,Product,Signature,SignatureStatus,Signed,DestinationHostname,DestinationIp,DestinationIsIpv6,DestinationPort,DestinationPortName,Initiated,Protocol,SourceHostname,SourceIp,SourceIsIpv6,SourcePort,SourcePortName,NewSd,OldSd,PipeName,CurrentDirectory,IntegrityLevel,LogonId,ParentCommandLine,ParentImage,ParentProcessGuid,ParentProcessId,TerminalSessionId,CallTrace,GrantedAccess,SourceProcessGUID,SourceThreadId,TargetProcessGUID,Device,TargetObject,Details,Application,FilterRTID,LayerRTID,SourceAddress,DestAddress,DestPort,Direction,RemoteMachineID,RemoteUserID,BootMenuPolicy,NewValue,NewValueType,ObjectValueName,OldValue,OldValueType,OperationType,ProfileUsed,RuleId,QueryName,QueryResults,QueryStatus,PreviousCreationUtcTime,EventCountTotal,EventIdx,GroupMembership,param1,Properties,Hash,PreAuthType,ClientProcessId,CountOfCredentialsReturned,ProcessCreationTime,ReadOperation,TargetName,Type,CalloutId,CalloutType,CorruptionActionState,DeviceName,DriveName,Config,IsTestConfig,DirtyPages,HiveName,HiveNameLength,KeysUpdated,PackageName,Workstation,DeviceNameLength,DeviceTime,DeviceVersionMajor,DeviceVersionMinor,FinalStatus,UserData,ServiceAccount,ServiceFileName,ServiceStartType,ServiceType,param10,param11,param2,param3,param4,param5,param6,param7,param8,param9,...,TargetLogonGuid,TargetServerName,ClientCreationTime,TimeSource,ImagePath,StartType,ProfileChanged,BootType,LoadOptions,EntryCount,KeyFilePath,ProviderContextKey,ProviderContextName,ProviderContextType,DwordVal,InterfaceGuid,InterfaceName,NewProfile,OldProfile,InstanceId,MaxRunspaces,MinRunspaces,CurrentStratumNumber,TimeSourceRefId,ReasonForRejection,Active,EdgeTraversal,EmbeddedContext,Flags,LocalAddresses,LocalOnlyMapped,LooseSourceMapped,ModifyingApplication,ModifyingUser,Origin,Profiles,RemoteAddresses,RuleStatus,SchemaVersion,SecurityOptions,CountNew,CountOld,EnabledNew,UpdateReason,NewTime,PreviousTime,NotificationPackageName,ShutdownActionType,ShutdownEventCode,ShutdownReason,BootMode,BuildVersion,MajorVersion,MinorVersion,QfeVersion,ServiceVersion,StartTime,EnableDisableReason,VsmPolicy,DCName,ProcessingMode,ProcessingTimeInMilliseconds,SupportInfo1,SupportInfo2,Gr

In [7]:
df_label.tail(1)

,@timestamp,@version,Category,Channel,EventID,EventReceivedTime,EventTime,EventType,ExecutionProcessID,Hostname,IpAddress,IpPort,Keywords,LogonGuid,Message,Opcode,OpcodeValue,ProviderGuid,RecordNumber,ServiceName,ServiceSid,Severity,SeverityValue,SourceModuleName,SourceModuleType,SourceName,Status,TargetDomainName,TargetUserName,Task,ThreadID,TicketEncryptionType,TicketOptions,TransmittedServices,Version,host,port,tags,TP_Label,Action,ActivityID,CalloutKey,CalloutName,ChangeType,Conditions,FilterId,FilterKey,FilterName,FilterType,LayerId,LayerKey,LayerName,ProcessId,ProviderKey,ProviderName,UserName,UserSid,Weight,AccessList,AccessMask,AccessReason,HandleId,ObjectName,ObjectServer,ObjectType,PrivilegeList,ProcessName,ResourceAttributes,RestrictedSidCount,SubjectDomainName,SubjectLogonId,SubjectUserName,SubjectUserSid,TransactionId,ShareLocalPath,ShareName,RelativeTargetName,CommandLine,MandatoryLabel,NewProcessId,NewProcessName,ParentProcessName,TargetLogonId,TargetUserSid,TokenElevationType,Service,DisabledPrivilegeList,EnabledPrivilegeList,CallerProcessId,CallerProcessName,TargetSid,AuthenticationPackageName,ElevatedToken,ImpersonationLevel,KeyLength,LmPackageName,LogonProcessName,LogonType,RestrictedAdminMode,TargetLinkedLogonId,TargetOutboundDomainName,TargetOutboundUserName,VirtualAccount,WorkstationName,SourceHandleId,SourceProcessId,TargetHandleId,TargetProcessId,AccountName,AccountType,ContextInfo,Domain,Payload,UserID,NewThreadId,RuleName,SourceImage,SourceProcessGuid,StartAddress,StartFunction,StartModule,TargetImage,TargetProcessGuid,UtcTime,MessageNumber,MessageTotal,Path,ScriptBlockId,ScriptBlockText,AlgorithmName,KeyName,KeyType,Operation,ReturnCode,Archived,Hashes,Image,IsExecutable,ProcessGuid,TargetFilename,User,CreationUtcTime,Company,Description,FileVersion,ImageLoaded,OriginalFileName,Product,Signature,SignatureStatus,Signed,DestinationHostname,DestinationIp,DestinationIsIpv6,DestinationPort,DestinationPortName,Initiated,Protocol,SourceHostname,SourceIp,SourceIsIpv6,SourcePort,SourcePortName,NewSd,OldSd,PipeName,CurrentDirectory,IntegrityLevel,LogonId,ParentCommandLine,ParentImage,ParentProcessGuid,ParentProcessId,TerminalSessionId,CallTrace,GrantedAccess,SourceProcessGUID,SourceThreadId,TargetProcessGUID,Device,TargetObject,Details,Application,FilterRTID,LayerRTID,SourceAddress,DestAddress,DestPort,Direction,RemoteMachineID,RemoteUserID,BootMenuPolicy,NewValue,NewValueType,ObjectValueName,OldValue,OldValueType,OperationType,ProfileUsed,RuleId,QueryName,QueryResults,QueryStatus,PreviousCreationUtcTime,EventCountTotal,EventIdx,GroupMembership,param1,Properties,Hash,PreAuthType,ClientProcessId,CountOfCredentialsReturned,ProcessCreationTime,ReadOperation,TargetName,Type,CalloutId,CalloutType,CorruptionActionState,DeviceName,DriveName,Config,IsTestConfig,DirtyPages,HiveName,HiveNameLength,KeysUpdated,PackageName,Workstation,DeviceNameLength,DeviceTime,DeviceVersionMajor,DeviceVersionMinor,FinalStatus,UserData,ServiceAccount,ServiceFileName,ServiceStartType,ServiceType,param10,param11,param2,param3,param4,param5,param6,param7,param8,param9,...,TargetLogonGuid,TargetServerName,ClientCreationTime,TimeSource,ImagePath,StartType,ProfileChanged,BootType,LoadOptions,EntryCount,KeyFilePath,ProviderContextKey,ProviderContextName,ProviderContextType,DwordVal,InterfaceGuid,InterfaceName,NewProfile,OldProfile,InstanceId,MaxRunspaces,MinRunspaces,CurrentStratumNumber,TimeSourceRefId,ReasonForRejection,Active,EdgeTraversal,EmbeddedContext,Flags,LocalAddresses,LocalOnlyMapped,LooseSourceMapped,ModifyingApplication,ModifyingUser,Origin,Profiles,RemoteAddresses,RuleStatus,SchemaVersion,SecurityOptions,CountNew,CountOld,EnabledNew,UpdateReason,NewTime,PreviousTime,NotificationPackageName,ShutdownActionType,ShutdownEventCode,ShutdownReason,BootMode,BuildVersion,MajorVersion,MinorVersion,QfeVersion,ServiceVersion,StartTime,EnableDisableReason,VsmPolicy,DCName,ProcessingMode,ProcessingTimeInMilliseconds,SupportInfo1,SupportInfo2,Gr

## Normalize Timestamp, IP/Port Columns
------------------

In [8]:
df_label['time_stamp'] = df_label['@timestamp'].apply(lambda x: (pd.Timestamp(x)))

In [9]:
df_label['time_stamp'] = df_label['time_stamp'].fillna(df_label['ts'].apply(lambda x: (pd.to_datetime(x, unit='s', utc=True))))

In [10]:
# Transform Host Data - Similar Data to Single Columns
df_label['Destination_IP'] = df_label['DestinationIp']
df_label['Destination_IP'] = df_label['Destination_IP'].fillna(df_label['DestAddress'])
df_label['Source_IP'] = df_label['IpAddress']
df_label['Source_IP'] = df_label['Source_IP'].fillna(df_label['SourceIp'])
df_label['Source_IP'] = df_label['Source_IP'].fillna(df_label['SourceAddress'])
df_label['Source_Port'] = df_label['IpPort']
df_label['Source_Port'] = df_label['Source_Port'].fillna(df_label['SourcePort'])
df_label['Destination_Port'] = df_label['DestinationPort']
df_label['Destination_Port'] = df_label['Destination_Port'].fillna(df_label['DestPort'])

In [11]:
# Transform Network Data - Similar Data to Single Columns
df_label['Source_IP'] = df_label['Source_IP'].fillna(df_label['id_orig_h'])
df_label['Source_Port'] = df_label['Source_Port'].fillna(df_label['id_orig_p'])
df_label['Destination_IP'] = df_label['Destination_IP'].fillna(df_label['id_resp_h'])
df_label['Destination_Port'] = df_label['Destination_Port'].fillna(df_label['id_resp_p'])

## Beagle Column Formatting

In [12]:
df_label['EventData_ParentImage'] = df_label['ParentImage'].astype(object)
df_label['EventData_ParentProcessId'] = df_label['ParentProcessId']
df_label['EventData_ParentProcessGuid'] = df_label['ParentProcessGuid']
df_label['EventData_Image'] = df_label['Image'].astype(object)
df_label['EventData_CommandLine'] = df_label['CommandLine'].astype(object)
df_label['EventData_Hashes'] = df_label['Hashes'].astype(object)

df_label['Computer'] = df_label['Hostname'].astype(object)
df_label['EventData_User'] = df_label['User'].astype(object)
df_label['EventData_UtcTime'] = df_label['UtcTime']

df_label['EventData_ProcessGuid'] = df_label['ProcessGuid']
df_label['EventData_ProcessId'] = df_label['ProcessId']

df_label['EventData_DestinationIp'] = df_label['DestinationIp']
df_label['EventData_DestinationPort'] = df_label['DestinationPort']
df_label['EventData_Protocol'] = df_label['Protocol']
df_label['EventData_TargetFilename'] = df_label['TargetFilename']

df_label['EventData_TargetObject'] = df_label['TargetObject']
df_label['EventData_EventType'] = df_label['EventType']
#df_label['EventData_Details'] = df_label['ProcessId']

## Manual Recurse Ancestor Chain Functions

In [13]:
def eval_ancestor_chain(G, starting_guid):
    for (p, d) in G.nodes(data=True):
        if 'process_guid' in d['properties'].keys():
            if starting_guid in d['properties']['process_guid']:
                selected_node = p
                
    #ancestors = nx.ancestors(G, selected_node)
    malicious_ancestors = [selected_node]
    for i in nx.ancestors(G, selected_node):
        if(G.node[i]['_node_type'] == 'Process'):
            print("==========================================")
            print("Node Data:")
            print("==========================================")
            print(G.node[i])
            print("==========================================")
            answer = None
            while answer not in ("yes", "no"):
                answer = input("Malicious? Enter yes or no: ")
                if answer == "yes":
                    # Do this.
                    malicious_ancestors.append(i)
                elif answer == "no":
                    # Do that.
                    return malicious_ancestors
                else:
                    print("Please enter yes or no.")
    return malicious_ancestors

def determine_malicious_root_proc(G, starting_guid):
    malicious_ancestors = eval_ancestor_chain(G, starting_guid)
    print("")
    print("##########################################")
    print("Confirm this as the senior malicious node?")
    print("##########################################")
    print("Node Data:")
    print("==========================================")
    print(G.node[malicious_ancestors[-1]])
    print("==========================================")
    print("Edge Out Data:")
    print("==========================================")
    for i in G.out_edges(malicious_ancestors[-1], data=True):
        if i[2]['type'] == 'Launched':
            print(i[2]['type'])
            print(G.node[i[1]])
            print("==========================================")
    answer = None
    while answer not in ("yes", "no"):
        answer = input("Malicious? Enter yes or no: ")
        if answer == "yes":
            # Do this.
            return malicious_ancestors[-1]
        elif answer == "no":
            # Do that.
            print("Re-run the eval_ancestor chain")
            malicious_root_ancestor = determine_malicious_root_proc(G, starting_guid)
            return malicious_root_ancestor
        else:
            print("Please enter yes or no.")
            
def get_ancestor_descendant_graph(G, malicious_root_ancestor):
    print(malicious_root_ancestor)

    descendants = nx.descendants(G, malicious_root_ancestor) #Returns all nodes reachable from source in G.
    print(len(descendants))

    ancestors = nx.ancestors(G, malicious_root_ancestor)
    print(len(ancestors))

    suspected_malicious = list(ancestors) + list(descendants)
    suspected_malicious.append(malicious_root_ancestor)
    print(len(suspected_malicious))

    H = G.subgraph(suspected_malicious)
    return H, malicious_root_ancestor, ancestors, descendants

def print_ancestor_descendant_graph(H, malicious_root_ancestor, ancestors, descendants):
    from matplotlib.pyplot import figure
    # Why are there orphaned nodes? I forgot to add the OG node. Fixed.
    # https://networkx.github.io/documentation/networkx-1.9/examples/drawing/labels_and_colors.html
    #fig = plt.Figure()
    figure(figsize=(15, 12))
    pos=nx.kamada_kawai_layout(H) # positions for all nodes

    # nodes
    nx.draw_networkx_nodes(H,pos,
                       nodelist=list(ancestors),
                       node_color='r',
                       #node_size=500,
                       label='ancestors',
                   alpha=0.8)
    nx.draw_networkx_nodes(H,pos,
                       nodelist=list(descendants),
                       node_color='b',
                       #node_size=500,
                       label='descendants',
                   alpha=0.8)
    nx.draw_networkx_nodes(H,pos,
                       nodelist=[malicious_root_ancestor],
                       node_color='g',
                       #node_size=500,
                       label='starting-point',
                   alpha=0.8)

    # edges
    nx.draw_networkx_edges(H,pos,width=1.0,alpha=0.5)

    #plt.legend()
    plt.legend(loc='upper left')
    #fig.update_layout(legend=dict(x=-.1, y=1.2))
    plt.axis('off')
    print("Saving as: ancestor_descendants_" + str(malicious_root_ancestor) + ".png")
    plt.savefig("ancestor_descendants_" + str(malicious_root_ancestor) + ".png") # save as png
    plt.show() # display
    
def get_ancestor_descendant_types(H):
    file_nodes = []
    for (p, d) in H.nodes(data=True):
        if d['_node_type']=='File':
            file_nodes.append(p)
    len(file_nodes)

    proc_nodes = []
    for (p, d) in H.nodes(data=True):
        if d['_node_type']=='Process':
            proc_nodes.append(p)
    len(proc_nodes)

    ip_nodes = []
    for (p, d) in H.nodes(data=True):
        if d['_node_type']=='IP Address':
            ip_nodes.append(p)
    len(ip_nodes)
    
    return file_nodes, proc_nodes, ip_nodes

def print_ancestor_descendant_type_graph(H, malicious_root_ancestor, file_nodes, proc_nodes, ip_nodes):
    # Why are there orphaned nodes? I forgot to add the OG node. Fixed.
    # https://networkx.github.io/documentation/networkx-1.9/examples/drawing/labels_and_colors.html
    #fig = plt.Figure()
    figure(figsize=(15, 12))
    pos=nx.kamada_kawai_layout(H) # positions for all nodes

    # nodes
    nx.draw_networkx_nodes(H,pos,
                           nodelist=file_nodes,
                           node_color='r',
                           #node_size=500,
                           label='file_nodes',
                       alpha=0.8)
    nx.draw_networkx_nodes(H,pos,
                           nodelist=proc_nodes,
                           node_color='b',
                           #node_size=500,
                           label='proc_nodes',
                       alpha=0.8)
    nx.draw_networkx_nodes(H,pos,
                           nodelist=ip_nodes,
                           node_color='g',
                           #node_size=500,
                           label='ip_nodes',
                       alpha=0.8)

    # edges
    nx.draw_networkx_edges(H,pos,width=1.0,alpha=0.5)

    #plt.legend()
    plt.legend(loc='upper left')
    #fig.update_layout(legend=dict(x=-.1, y=1.2))
    plt.axis('off')
    print("Saving as: ancestor_descendants_nodeType_" + str(malicious_root_ancestor) + ".png")
    plt.savefig("ancestor_descendants_nodeType_" + str(malicious_root_ancestor) + ".png") # save as png
    plt.show() # display
    
def analyze_graph_vs_label_proc(H, proc_nodes):
    graph_bad_guids = []
    for i in proc_nodes:
        graph_bad_guids.append(H.node[i]['properties']['process_guid'])
    print(len(graph_bad_guids))
    #print(graph_bad_guids)
    
    label_bad_guids = df_label.loc[(df_label['TP_Binary'] == True) & (df_label.ProcessGuid.notnull())]['ProcessGuid'].unique()
    pd_all_guids = df_label.loc[(df_label.ProcessGuid.notnull()) & (df_label.CommandLine.notnull())][['ProcessGuid', 'Hostname', 'TP_Binary', 'CommandLine']]
    pd_all_guids['orig_label'] = False
    pd_all_guids['orig_label'].loc[pd_all_guids.ProcessGuid.isin(label_bad_guids)] = True

    pd_all_guids = pd_all_guids.drop(columns=['TP_Binary']).drop_duplicates()
    pd_all_guids['graph_label'] = False
    pd_all_guids['graph_label'].loc[pd_all_guids.ProcessGuid.isin(graph_bad_guids)] = True

    with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):
        display(pd_all_guids[['ProcessGuid', 'Hostname', 'orig_label', 'graph_label', 'CommandLine']].loc[((pd_all_guids.orig_label==True) | (pd_all_guids.graph_label==True))].sort_values(by=['orig_label', 'graph_label'], ascending=False))
        
def manual_recurse_pipeline(G, starting_point):
    malicious_root_ancestor = determine_malicious_root_proc(G, starting_point)
    print(malicious_root_ancestor)
    
    H, malicious_root_ancestor, ancestors, descendants = get_ancestor_descendant_graph(G, malicious_root_ancestor)
    
    #print_ancestor_descendant_graph(H, malicious_root_ancestor, ancestors, descendants)
    
    file_nodes, proc_nodes, ip_nodes = get_ancestor_descendant_types(H)
    
    #print_ancestor_descendant_type_graph(H, malicious_root_ancestor, file_nodes, proc_nodes, ip_nodes)
    
    #analyze_graph_vs_label_proc(H, proc_nodes)
    
    return H, malicious_root_ancestor, ancestors, descendants, file_nodes, proc_nodes, ip_nodes


# Testing Beagle

In [14]:
from beagle.backends import NetworkX
from beagle.transformers import SysmonTransformer

In [15]:
import os
import json
from typing import Dict, Generator, List

from beagle.datasources.base_datasource import DataSource
from beagle.transformers import GenericTransformer
from beagle.transformers.sysmon_transformer import SysmonTransformer

class JSONData(DataSource):
    """A generic data source which returns events one by one
    """

    name = "JSON Data"
    transformers = [SysmonTransformer]
    category = "Generic Data"

    def __init__(self, events: List[Dict]) -> None:
        self._events = events

    def events(self) -> Generator[dict, None, None]:
        for event in self._events:
            yield event

    def metadata(self) -> dict:
        return {}

In [32]:
from beagle.backends import NetworkX

NX = NetworkX.from_datasources(
    datasources=[
        JSONData(df_label.loc[(df_label['EventID']==1)].to_dict('records')),
        JSONData(df_label.loc[(df_label['EventID']==3)].to_dict('records')),
        JSONData(df_label.loc[(df_label['EventID']==11)].to_dict('records')),
    ]
)
G = NX.graph()

2020-07-04T07:12:51 | beagle.transformers.sysmon_transformer.__init__:15 | INFO | Created Sysmon Transformer.
2020-07-04T07:12:51 | beagle.transformers.base_transformer._consumer_thread:143 | WARNING | Error when parsing event, recieved exception 'float' object has no attribute 'split'
2020-07-04T07:12:51 | beagle.transformers.base_transformer._consumer_thread:143 | WARNING | Error when parsing event, recieved exception 'float' object has no attribute 'split'
2020-07-04T07:12:51 | beagle.transformers.base_transformer._consumer_thread:143 | WARNING | Error when parsing event, recieved exception 'float' object has no attribute 'split'
2020-07-04T07:12:51 | beagle.transformers.base_transformer._consumer_thread:143 | WARNING | Error when parsing event, recieved exception 'float' object has no attribute 'split'
2020-07-04T07:12:51 | beagle.transformers.base_transformer.run:111 | INFO | Finished processing of events, created 1784 nodes.
2020-07-04T07:12:51 | beagle.transformers.base_transfor

In [33]:
G

In [34]:
graph_json = NetworkX.graph_to_json(G)

2020-07-04T07:12:56 | beagle.backends.networkx.__init__:56 | INFO | Initialized NetworkX Backend


In [35]:
from networkx.readwrite import json_graph

G = json_graph.node_link_graph(graph_json)

In [37]:
G = nx.convert_node_labels_to_integers(G, first_label=0, ordering='default', label_attribute='Orig_Node_ID')

# Large Malicious Subgraph

In [38]:
# Now determine the ancestor/descendant nodes around selected_node
M1, malicious_root_ancestor, ancestors, descendants, file_nodes, proc_nodes, ip_nodes = manual_recurse_pipeline(G, '{47ab858c-e13c-5eac-a903-000000000400}')

Node Data:
{'properties': {'process_guid': '{47ab858c-dac4-5eac-f202-000000000400}', 'host': 'SCRANTON.dmevals.local', 'process_id': 4440, 'user': nan, 'process_image': 'Explorer.EXE', 'process_image_path': 'C:\\windows', 'command_line': None, 'hashes': {}, 'process_path': 'C:\\windows\\Explorer.EXE'}, '_node_type': 'Process', '_node_class': 'SysMonProc', '_display': 'Explorer.EXE', '_color': '#FF0000', 'Orig_Node_ID': -5417281893827090961}


Malicious? Enter yes or no:  no



##########################################
Confirm this as the senior malicious node?
##########################################
Node Data:
{'properties': {'process_guid': '{47ab858c-e13c-5eac-a903-000000000400}', 'host': 'SCRANTON.dmevals.local', 'process_id': 8524, 'user': nan, 'process_image': 'â€®cod.3aka3.scr', 'process_image_path': 'C:\\ProgramData\\victim', 'command_line': '"C:\\ProgramData\\victim\\â€®cod.3aka3.scr" /S', 'hashes': {'sha1': '4B7FA56A4E85F88B98D11A6E018698AE3FBA5E62', 'md5': '9D1C5EF38E6073661C74660B3A71A76E', 'sha256': '0DF38A55D940F498478EB03683C94D4584236E100125B526A67650BA54DF4AE4', 'imphash': 'F00447512A354E59D39D2818AABA4A17'}, 'process_path': 'C:\\ProgramData\\victim\\â€®cod.3aka3.scr'}, '_node_type': 'Process', '_node_class': 'SysMonProc', '_display': 'â€®cod.3aka3.scr', '_color': '#FF0000', 'Orig_Node_ID': 3685170341351964961}
Edge Out Data:
Launched
{'properties': {'process_guid': '{47ab858c-e188-5eac-af03-000000000400}', 'host': 'SCRANTON.dmevals.loca

Malicious? Enter yes or no:  yes


1365
1365
489
4
494


# Clustering

In [44]:
H = G.subgraph(max(nx.weakly_connected_components(G), key=len))

In [45]:
H

In [52]:
# Degree Centrality
c_degree = nx.degree_centrality(H)
len(c_degree)

1975

In [53]:
# Average Neighbor Degree
an_degree = nx.algorithms.assortativity.average_neighbor_degree(H)
len(an_degree)
#an_degree

1975

In [46]:
import networkx as nx
import community
import matplotlib.pyplot as plt

In [48]:
# Best Partition
import community
# requires undirected graph
I = H.to_undirected()

best_partition = community.best_partition(I)  # compute communities
len(best_partition)

1975

In [51]:
# Clustering Coefficient
J = nx.DiGraph(H)

# requires DiGraph (instead of MultiDiGraph)
cluster_coef = nx.algorithms.cluster.clustering(J)
len(cluster_coef)

1975

In [68]:
# Greedy Modularity Communities
# Now theres 33 communities from the Greedy_Mode -> is greedy_mod effected by attributes? 
#     -> Only change was adding the betweenness attribute?
#     -> difference between directed and undirected? (H) directed doesn't compute here...
#     -> trying greedy mod on G -> apply to I -> nope. I -> I works
#     -> trying DiGraph (J) -> apply to I
from networkx.algorithms import community
communities = community.greedy_modularity_communities(I)
print(f"The graph has {len(communities)} communities.")

modularity_dict = {} # Create a blank dictionary
for i,c in enumerate(communities): # Loop through the list of communities, keeping track of the number for the community
    for name in c: # Loop through each person in a community
        modularity_dict[name] = i # Create an entry in the dictionary for the person, where the value is which group they belong to.

# Now you can add modularity information like we did the other metrics
nx.set_node_attributes(I, modularity_dict, 'Greedy_Mod')

The graph has 33 communities.


In [58]:
# Betweenness
# https://programminghistorian.org/en/lessons/exploring-and-analyzing-network-data-with-python
betweenness_dict = nx.betweenness_centrality(I) # Run betweenness centrality

#PowerIterationFailedConvergence: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 100 iterations')
#eigenvector_dict = nx.eigenvector_centrality(L) # Run eigenvector centrality

# Assign each to an attribute in your network
nx.set_node_attributes(I, betweenness_dict, 'betweenness')
#nx.set_node_attributes(L, eigenvector_dict, 'eigenvector')

In [72]:
# Label Propagation
# Trying H -> I  nope. Not implemented for directed type
# Trying I -> I
# Quick.
communities = community.label_propagation_communities(I)
# <generator object label_propagation_communities at 0x7f7858f98450>

label_prop_dict = {} # Create a blank dictionary
for i,c in enumerate(communities): # Loop through the list of communities, keeping track of the number for the community
    for name in c: # Loop through each person in a community
        label_prop_dict[name] = i # Create an entry in the dictionary for the person, where the value is which group they belong to.

# Now you can add modularity information like we did the other metrics
nx.set_node_attributes(I, label_prop_dict, 'label_prop')

## Cluster Analysis - DataFrame

In [80]:
df = pd.DataFrame.from_dict(dict(I.nodes(data=True)), orient='index')
df = pd.concat([df.drop(['properties'], axis=1), df['properties'].apply(pd.Series)], axis=1)
df = df.reset_index().rename(columns={"index" : "NodeID"})
df['DegreeCentrality'] =  df['NodeID'].apply(lambda x: (c_degree[x]))
df['ClusterCoef'] =  df['NodeID'].apply(lambda x: (cluster_coef[x]))
df['BestPartition'] = df['NodeID'].apply(lambda x: (best_partition[x]))
df['AveNeighDegree'] = df['NodeID'].apply(lambda x: (an_degree[x]))

df.head()

,NodeID,_node_type,_node_class,_display,_color,Orig_Node_ID,betweenness,Greedy_Mod,label_prop,process_guid,host,process_id,user,process_image,process_image_path,command_line,hashes,process_path,file_path,file_name,full_path,extension,ip_address,mac,DegreeCentrality,ClusterCoef,BestPartition,AveNeighDegree
0,0,Process,SysMonProc,svchost.exe,#FF0000,-338401461626367812,0.227097,11,108,{47ab858c-cadc-5eac-2200-000000000400},SCRANTON.dmevals.local,1448.0,None,svchost.exe,C:\Windows\System32,None,{},C:\Windows\System32\svchost.exe,NaN,NaN,NaN,NaN,NaN,NaN,0.003546,0.0,0,55.833333
1,1,Process,SysMonProc,taskhostw.exe,#FF0000,1951897641756709011,0.000487,11,108,{47ab858c-e589-5eac-f803-000000000400},SCRANTON.dmevals.local,8288.0,DMEVALS\pbeesly,taskhostw.exe,C:\Windows\System32,taskhostw.exe Install $(Arg0),"{'sha1': '9245F36AA783167E0AFE948B1C0E8E7F3B81E3D4', 'md5': '8AC325C757FA721B272ECEA19EBCF745', 'sha256': '0BEC3128B241FA533DC5A9D3F62AA41323D205DEFC30DCDE228C5ECC2D1E50A9', 'imphash': '9CB27CAED52CB0AFFB32788922A0D083'}",C:\Windows\System32\taskhostw.exe,NaN,NaN,NaN,NaN,NaN,NaN,0.001013,0.0,0,0.000000
2,2,Process,SysMonProc,gpupdate.exe,#FF0000,5028989411627948319,0.058282,11,143,{47ab858c-e296-5eac-c803-000000000400},SCRANTON.dmevals.local,760.0,NT AUTHORITY\NETWORK SERVICE,gpupdate.exe,C:\Windows\System32,gpupdate.exe /target:computer,"{'sha1': '1529CB7F7DFA798337F781A71B583371E1D43F2B', 'md5': 'D4F01BEC9CA921C13ED1BDD1BF1D2D24', 'sha256': 'FE428F64B6920CBD542BF7097F009A576673888967CB5AE8803D310667ED428D', 'imphash': '874ADC3991B152C2F8C8ABC6E3F65A5C'}",C:\Windows\System32\gpupdate.exe,NaN,NaN,NaN,NaN,NaN,NaN,0.001520,0.0,0,0.000000
3,3,Process,SysMonProc,taskhostw.exe,#FF0000,-7720460255082120347,0.000487,11,108,{47ab858c-e2d4-5eac-cd03-000000000400},SCRANTON.dmevals.local,8532.0,DMEVALS\pbeesly,taskhostw.exe,C:\Windows\System32,taskhostw.exe Install $(Arg0),"{'sha1': '9245F36AA783167E0AFE948B1C0E8E7F3B81E3D4', 'md5': '8AC325C757FA721B272ECEA19EBCF745', 'sha256': '0BEC3128B241FA533DC5A9D3F62AA41323D205DEFC30DCDE228C5ECC2D1E50A9', 'imphash': '9CB27CAED52CB0AFFB32788922A0D083'}",C:\Windows\System32\taskhostw.exe,NaN,NaN,NaN,NaN,NaN,NaN,0.001013,0.0,0,0.000000
4,4,Process,SysMonProc,taskhostw.exe,#FF0000,6980313586955071755,0.000487,11,108,{47ab858c-e37a-5eac-e003-000000000400},SCRANTON.dmevals.local,1548.0,DMEVALS\pbeesly,taskhostw.exe,C:\Windows\System32,taskhostw.exe Event,"{'sha1': '9245F36AA783167E0AFE948B1C0E8E7F3B81E3D4', 'md5': '8AC325C757FA721B272ECEA19EBCF745', 'sha256': '0BEC3128B241FA533DC5A9D3F62AA41323D205DEFC30DCDE228C5ECC2D1E50A9', 'imphash': '9CB27CAED52CB0AFFB32788922A0D083'}",C:\Windows\System32\taskhostw.exe,NaN,NaN,NaN,NaN,NaN,NaN,0.001013,0.0,0,0.000000


In [81]:
df['Chain_Loc'] = np.nan
df['Chain_Loc'].loc[df.NodeID.isin(list(ancestors))] = '1_Ancestor' #[['_display', 'DegreeCentrality']]
df['Chain_Loc'].loc[df.NodeID.isin([malicious_root_ancestor])] = '2_Root' #[['_display', 'DegreeCentrality']]
df['Chain_Loc'].loc[df.NodeID.isin(list(descendants))] = '3_Descendant' #[['_display', 'DegreeCentrality']]
df.loc[((df['Chain_Loc']=='1_Ancestor') | (df['Chain_Loc']=='2_Root') | ((df['Chain_Loc']=='3_Descendant') & (df['_node_type']=='Process')))].sort_values(by=['Chain_Loc'], ascending=True)[['_display', '_node_type', 'Chain_Loc', 'BestPartition', 'Greedy_Mod', 'label_prop', 'process_path', 'betweenness', 'DegreeCentrality', 'ClusterCoef', 'AveNeighDegree']]

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,_display,_node_type,Chain_Loc,BestPartition,Greedy_Mod,label_prop,process_path,betweenness,DegreeCentrality,ClusterCoef,AveNeighDegree
1954,Explorer.EXE,File,1_Ancestor,13,9,146,NaN,0.000161,0.001013,0.000000,7.500000
1344,â€®cod.3aka3.scr,File,1_Ancestor,11,4,101,NaN,0.000000,0.000507,0.000000,6.000000
1434,Explorer.EXE,Process,1_Ancestor,13,9,146,C:\windows\Explorer.EXE,0.009953,0.005572,0.000000,0.666667
1433,explorer.exe,File,1_Ancestor,13,9,146,NaN,0.000161,0.001013,0.000000,7.500000
1342,â€®cod.3aka3.scr,Process,2_Root,11,4,101,C:\ProgramData\victim\â€®cod.3aka3.scr,0.013714,0.004053,0.000000,1.000000
1238,cvtres.exe,Process,3_Descendant,21,5,90,C:\Windows\Microsoft.NET\Framework64\v4.0.30319\cvtres.exe,0.000783,0.001520,0.166667,0.000000
1235,cvtres.exe,Process,3_Descendant,21,5,63,C:\Windows\Microsoft.NET\Framework64\v4.0.30319\cvtres.exe,0.000232,0.001520,0.166667,0.000000
1061,control.exe,Process,3_Descendant,11,5,7,C:\Windows\System32\control.exe,0.002369,0.001520,0.000000,19.000000
1060,sdclt.exe,Process,3_Descendant,11,4,8,C:\Windows\System32\sdclt.exe,0.000122,0.001013,0.000000,0.000000
1059,sdclt.exe,Process,3_Descendant,11,4,8,C:\Windows\System32\sdclt.exe,0.000122,0.001013,0.000000,0.000000


In [86]:
df['Chain_Loc'] = df['Chain_Loc'].fillna(False)
df['Chain_Loc_Binary'] = df['Chain_Loc']!=False

## Greedy_Mod Evaluation

In [87]:
#df.loc[(df['Greedy_Mod'].isin([9, 4, 5, 0]))].groupby()
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df.loc[df.Greedy_Mod.isin([9, 4, 5, 0])].groupby(['Chain_Loc_Binary', 'Greedy_Mod']).count())

NodeID  _node_type  _node_class  _display  \
Chain_Loc_Binary Greedy_Mod                                              
False            0                3           3            3         3   
                 4              141         141          141       141   
                 5                7           7            7         7   
                 9               53          53           53        53   
True             0              324         324          324       324   
                 4               33          33           33        33   
                 5              131         131          131       131   
                 9                3           3            3         3   

                             _color  Orig_Node_ID  betweenness  label_prop  \
Chain_Loc_Binary Greedy_Mod                                                  
False            0                3             3            3           3   
                 4              141           141          141         141   
                 5                7             7            7           7   
                 9               53            53           53          53   
True             0              324           324          324         324   
                 4               33            33           33          33   
                 5              131           131          131         131   
                 9                3             3            3           3   

                             process_guid  host  process_id  user  \
Chain_Loc_Binary Greedy_Mod                                         
False            0                      0     3           0     0   
                 4                     95   124          95    87   
                 5                      1     7           1     1   
                 9                     25    39          25    21   
True             0                      5     5           5     3   
                 4                     16    17          16    10   
                 5                     13    13          13     7   
                 9                      1     3           1     0   

                             process_image  process_image_path  command_line  \
Chain_Loc_Binary Greedy_Mod                                                    
False            0                       0                   0             0   
                 4                      95                  95            93   
                 5                       1                   1             0   
                 9                      25                  25            24   
True             0                       5                   5             5   
                 4                      16                  16            16   
                 5                      13                  13            13   
                 9                       1                   1             0   

                             hashes  process_path  file_path  file_name  \
Chain_Loc_Binary Greedy_Mod                                               
False            0                3             0          3          3   
                 4              141            95         46         46   
                 5                7             1          6          6   
                 9               53            25         28         28   
True             0              324             5        319        319   
                 4               33            16         17         17   
                 5              126            13        113        113   
                 9                3             1          2          2   

                             full_path  extension  ip_address  mac  \
Chain_Loc_Binary Greedy_Mod                                          
False            0                   3          0           0    0   
                 4                  46          0     

In [92]:
# So Community 9 is the first label with a greater proportion in the False.
# Set(0, 4, 5) -> (9) also marks the malicious/benign boundary. 
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df.loc[(df['_node_type']=='Process') & (df.Greedy_Mod.isin([0, 4, 5, 9]))].groupby(['Chain_Loc_Binary', 'Greedy_Mod']).count())

NodeID  _node_type  _node_class  _display  \
Chain_Loc_Binary Greedy_Mod                                              
False            4               95          95           95        95   
                 5                1           1            1         1   
                 9               25          25           25        25   
True             0                5           5            5         5   
                 4               16          16           16        16   
                 5               13          13           13        13   
                 9                1           1            1         1   

                             _color  Orig_Node_ID  betweenness  label_prop  \
Chain_Loc_Binary Greedy_Mod                                                  
False            4               95            95           95          95   
                 5                1             1            1           1   
                 9               25            25           25          25   
True             0                5             5            5           5   
                 4               16            16           16          16   
                 5               13            13           13          13   
                 9                1             1            1           1   

                             process_guid  host  process_id  user  \
Chain_Loc_Binary Greedy_Mod                                         
False            4                     95    95          95    87   
                 5                      1     1           1     1   
                 9                     25    25          25    21   
True             0                      5     5           5     3   
                 4                     16    16          16    10   
                 5                     13    13          13     7   
                 9                      1     1           1     0   

                             process_image  process_image_path  command_line  \
Chain_Loc_Binary Greedy_Mod                                                    
False            4                      95                  95            93   
                 5                       1                   1             0   
                 9                      25                  25            24   
True             0                       5                   5             5   
                 4                      16                  16            16   
                 5                      13                  13            13   
                 9                       1                   1             0   

                             hashes  process_path  file_path  file_name  \
Chain_Loc_Binary Greedy_Mod                                               
False            4               95            95          0          0   
                 5                1             1          0          0   
                 9               25            25          0          0   
True             0                5             5          0          0   
                 4               16            16          0          0   
                 5               13            13          0          0   
                 9                1             1          0          0   

                             full_path  extension  ip_address  mac  \
Chain_Loc_Binary Greedy_Mod                                          
False            4                   0          0           0    0   
                 5                   0          0           0    0   
                 9                   0          0           0    0   
True             0                   0          0           0    0   
                 4                   0          0           0    0   
                 5                   0          0           0    0   
                 9                   0          0           0    0   

         

## Start at the bottom test - SCRANTON

In [93]:
# Lets try reverse order. Land at the bottom of the emulation plan. See if we can use the Greedy_Mod communities to recurse up.
# Last Threat Emulation Activity on SCRANTON in this graph.
#Lateral Movement	"Windows Admin Shares,Service Execution,Valid Accounts"
#8.C	This new payload is executed on the secondary victim via the PSExec utility (T1077, T1035) using the previously stolen credentials (T1078).	"Execute SEADUKE Remotely via PSEXEC
#[meterpreter (PowerShell)\*] >.\PsExec64.exe -accepteula \\NASHUA -u ""dmevals\pbeesly"" -p ""Fl0nk3rt0n!T0by"" -i <Session Number from step 8.A. e.g. 4> ""C:\Windows\Temp\python.exe"""	admin	SCRANTON	NASHUA

In [94]:
df.loc[((df['Chain_Loc']=='1_Ancestor') | (df['Chain_Loc']=='2_Root') | ((df['Chain_Loc']=='3_Descendant') & (df['_node_type']=='Process')))].sort_values(by=['Chain_Loc'], ascending=True)

,NodeID,_node_type,_node_class,_display,_color,Orig_Node_ID,betweenness,Greedy_Mod,label_prop,process_guid,host,process_id,user,process_image,process_image_path,command_line,hashes,process_path,file_path,file_name,full_path,extension,ip_address,mac,DegreeCentrality,ClusterCoef,BestPartition,AveNeighDegree,Chain_Loc,Chain_Loc_Binary
1954,2028,File,File,Explorer.EXE,#3CB371,-5533181356103591401,0.000161,9,146,NaN,SCRANTON.dmevals.local,NaN,NaN,NaN,NaN,NaN,{},NaN,C:\windows,Explorer.EXE,C:\windows\Explorer.EXE,NaN,NaN,NaN,0.001013,0.000000,13,7.500000,1_Ancestor,True
1344,1367,File,File,â€®cod.3aka3.scr,#3CB371,1951582023185893553,0.000000,4,101,NaN,SCRANTON.dmevals.local,NaN,NaN,NaN,NaN,NaN,"{'sha1': '4B7FA56A4E85F88B98D11A6E018698AE3FBA5E62', 'md5': '9D1C5EF38E6073661C74660B3A71A76E', 'sha256': '0DF38A55D940F498478EB03683C94D4584236E100125B526A67650BA54DF4AE4', 'imphash': 'F00447512A354E59D39D2818AABA4A17'}",NaN,C:\ProgramData\victim,â€®cod.3aka3.scr,C:\ProgramData\victim\â€®cod.3aka3.scr,NaN,NaN,NaN,0.000507,0.000000,11,6.000000,1_Ancestor,True
1434,1461,Process,SysMonProc,Explorer.EXE,#FF0000,-5417281893827090961,0.009953,9,146,{47ab858c-dac4-5eac-f202-000000000400},SCRANTON.dmevals.local,4440.0,NaN,Explorer.EXE,C:\windows,None,{},C:\windows\Explorer.EXE,NaN,NaN,NaN,NaN,NaN,NaN,0.005572,0.000000,13,0.666667,1_Ancestor,True
1433,1460,File,File,explorer.exe,#3CB371,3618043331945603986,0.000161,9,146,NaN,SCRANTON.dmevals.local,NaN,NaN,NaN,NaN,NaN,"{'sha1': 'C893CF07E5F65749CD66E17D9523638B132C87B2', 'md5': 'F7DC8A74E30E08B9510380274CFB9288', 'sha256': 'C5E88D778C0B118D49BEF467ED059C09B61DEEA505D2A3D5CA1DCC0A5CDF752F', 'imphash': 'FE6F775DD0C72FFD106F56930C60A452'}",NaN,C:\Windows,explorer.exe,C:\Windows\explorer.exe,NaN,NaN,NaN,0.001013,0.000000,13,7.500000,1_Ancestor,True
1342,1365,Process,SysMonProc,â€®cod.3aka3.scr,#FF0000,3685170341351964961,0.013714,4,101,{47ab858c-e13c-5eac-a903-000000000400},SCRANTON.dmevals.local,8524.0,NaN,â€®cod.3aka3.scr,C:\ProgramData\victim,"""C:\ProgramData\victim\â€®cod.3aka3.scr"" /S","{'sha1': '4B7FA56A4E85F88B98D11A6E018698AE3FBA5E62', 'md5': '9D1C5EF38E6073661C74660B3A71A76E', 'sha256': '0DF38A55D940F498478EB03683C94D4584236E100125B526A67650BA54DF4AE4', 'imphash': 'F00447512A354E59D39D2818AABA4A17'}",C:\ProgramData\victim\â€®cod.3aka3.scr,NaN,NaN,NaN,NaN,NaN,NaN,0.004053,0.000000,11,1.000000,2_Root,True
1238,1256,Process,SysMonProc,cvtres.exe,#FF0000,-2091218517300341226,0.000783,5,90,{47ab858c-e413-5eac-e903-000000000400},SCRANTON.dmevals.local,620.0,NaN,cvtres.exe,C:\Windows\Microsoft.NET\Framework64\v4.0.30319,"C:\Windows\Microsoft.NET\Framework64\v4.0.30319\cvtres.exe /NOLOGO /READONLY /MACHINE:IX86 ""/OUT:C:\Users\pbeesly\AppData\Local\Temp\RESAB37.tmp"" ""c:\Users\pbeesly\AppData\Local\Temp\0piklvia\CSC8FB4597E15924B708420DCB4F6EA7BB.TMP""","{'sha1': '2A3249732F5AA4588A4A9895FFE217355041D663', 'md5': 'C877CBB966EA5939AA2A17B6A5160950', 'sha256': '1FE531EAC592B480AA4BD16052B909C3431434F17E7AE163D248355558CE43A6', 'imphash': '55D76ADE7FFEA0F41FF2B55505C2B362'}",C:\Windows\Microsoft.NET\Framework64\v4.0.30319\cvtres.exe,NaN,NaN,NaN,NaN,NaN,NaN,0.001520,0.166667,21,0.000000,3_Descendant,True
1235,1253,Process,SysMonProc,cvtres.exe,#FF0000,7749550861269082083,0.000232,5,63,{47ab858c-e1e5-5eac-bb03-000000000400},SCRANTON.dmevals.local,8528.0,NaN,cvtres.exe,C:\Windows\Microsoft.NET\Framework64\v4.0.30319,"C:\Windows\Microsoft.NET\Framework64\v4.0.30319\cvtres.exe /NOLOGO /READONLY /MACHINE:IX86 ""/OUT:C:\Users\pbeesly\AppData\Local\Temp\RES2758.tmp"" ""c:\Users\pbeesly\AppData\Local\Temp\qkbkqqbs\CSC6B76613243A54474837EBEC19BC0E56A.TMP""","{'sha1': '2A3249732F5AA4588A4A9895FFE217355041D663', 'md5': 'C877CBB966EA5939AA2A17B6A5160950', 'sha256': '1FE531EAC592B480AA4BD16052B909C3431434F17E7AE163D248355558CE43A6', 'imphash': '55D76ADE7FFEA0F41FF2B55505C2B362'}",C:\Windows\Microsoft.NET\Framework64\v4.0.30319\cvtres.exe,NaN,NaN,NaN,NaN,NaN,NaN,0.001520,0.166667,21,0.000000,3_Descendant,True
1061,107

In [95]:
df.loc[df['process_image'] == 'PsExec64.exe']

,NodeID,_node_type,_node_class,_display,_color,Orig_Node_ID,betweenness,Greedy_Mod,label_prop,process_guid,host,process_id,user,process_image,process_image_path,command_line,hashes,process_path,file_path,file_name,full_path,extension,ip_address,mac,DegreeCentrality,ClusterCoef,BestPartition,AveNeighDegree,Chain_Loc,Chain_Loc_Binary
823,836,Process,SysMonProc,PsExec64.exe,#FF0000,-3091633888033844280,0.000253,5,100,{47ab858c-e519-5eac-f403-000000000400},SCRANTON.dmevals.local,5924.0,DMEVALS\pbeesly,PsExec64.exe,C:\Program Files\SysinternalsSuite,"""C:\Program Files\SysinternalsSuite\PsExec64.exe"" -accepteula \\NASHUA -u dmevals\pbeesly -p Fl0nk3rt0n!T0by -i 2 C:\Windows\Temp\python.exe","{'sha1': 'FB0A150601470195C47B4E8D87FCB3F50292BEB2', 'md5': '9321C107D1F7E336CDA550A2BF049108', 'sha256': 'AD6B98C01EE849874E4B4502C3D7853196F6044240D3271E4AB3FC6E3C08E9A4', 'imphash': '159D56D406180A332FBC99290F30700E'}",C:\Program Files\SysinternalsSuite\PsExec64.exe,NaN,NaN,NaN,NaN,NaN,NaN,0.001520,0.166667,8,0.0,3_Descendant,True
824,837,Process,SysMonProc,PsExec64.exe,#FF0000,6274138833682693877,0.000253,5,100,{47ab858c-e5a2-5eac-fb03-000000000400},SCRANTON.dmevals.local,8928.0,DMEVALS\pbeesly,PsExec64.exe,C:\Program Files\SysinternalsSuite,"""C:\Program Files\SysinternalsSuite\PsExec64.exe"" -accepteula \\NASHUA -u dmevals\pbeesly -p Fl0nk3rt0n!T0by -i 2 C:\Windows\Temp\python.exe","{'sha1': 'FB0A150601470195C47B4E8D87FCB3F50292BEB2', 'md5': '9321C107D1F7E336CDA550A2BF049108', 'sha256': 'AD6B98C01EE849874E4B4502C3D7853196F6044240D3271E4AB3FC6E3C08E9A4', 'imphash': '159D56D406180A332FBC99290F30700E'}",C:\Program Files\SysinternalsSuite\PsExec64.exe,NaN,NaN,NaN,NaN,NaN,NaN,0.002026,0.166667,8,0.0,3_Descendant,True
825,838,Process,SysMonProc,PsExec64.exe,#FF0000,-934027247007052661,0.000253,5,100,{47ab858c-e558-5eac-f503-000000000400},SCRANTON.dmevals.local,1412.0,DMEVALS\pbeesly,PsExec64.exe,C:\Program Files\SysinternalsSuite,"""C:\Program Files\SysinternalsSuite\PsExec64.exe"" -accepteula \\NASHUA -u dmevals\pbeesly -p Fl0nk3rt0n!T0by -i 2 C:\Windows\Temp\python.exe","{'sha1': 'FB0A150601470195C47B4E8D87FCB3F50292BEB2', 'md5': '9321C107D1F7E336CDA550A2BF049108', 'sha256': 'AD6B98C01EE849874E4B4502C3D7853196F6044240D3271E4AB3FC6E3C08E9A4', 'imphash': '159D56D406180A332FBC99290F30700E'}",C:\Program Files\SysinternalsSuite\PsExec64.exe,NaN,NaN,NaN,NaN,NaN,NaN,0.001520,0.166667,8,0.0,3_Descendant,True
826,839,Process,SysMonProc,PsExec64.exe,#FF0000,8130003157897699778,0.000253,5,100,{47ab858c-e4d6-5eac-f303-000000000400},SCRANTON.dmevals.local,2668.0,DMEVALS\pbeesly,PsExec64.exe,C:\Program Files\SysinternalsSuite,"""C:\Program Files\SysinternalsSuite\PsExec64.exe"" -accepteula \\NASHUA -u dmevals\pbeesly -p Fl0nk3rt0n!T0by -i 2 C:\Windows\Temp\python.exe","{'sha1': 'FB0A150601470195C47B4E8D87FCB3F50292BEB2', 'md5': '9321C107D1F7E336CDA550A2BF049108', 'sha256': 'AD6B98C01EE849874E4B4502C3D7853196F6044240D3271E4AB3FC6E3C08E9A4', 'imphash': '159D56D406180A332FBC99290F30700E'}",C:\Program Files\SysinternalsSuite\PsExec64.exe,NaN,NaN,NaN,NaN,NaN,NaN,0.001520,0.166667,8,0.0,3_Descendant,True


In [97]:
#df.loc[(df['Chain_Loc_Binary']==True) & (df['_node_type']=='Process')]

,NodeID,_node_type,_node_class,_display,_color,Orig_Node_ID,betweenness,Greedy_Mod,label_prop,process_guid,host,process_id,user,process_image,process_image_path,command_line,hashes,process_path,file_path,file_name,full_path,extension,ip_address,mac,DegreeCentrality,ClusterCoef,BestPartition,AveNeighDegree,Chain_Loc,Chain_Loc_Binary
186,186,Process,SysMonProc,conhost.exe,#FF0000,5187003371410181947,0.011129,5,85,{47ab858c-e374-5eac-d903-000000000400},SCRANTON.dmevals.local,7884.0,DMEVALS\pbeesly,conhost.exe,C:\Windows\System32,\??\C:\windows\system32\conhost.exe 0xffffffff -ForceV1,"{'sha1': '11996F32DD85863A8C3BFF6D520F788A9211C8F7', 'md5': 'C5E9B1D1103EDCEA2E408E9497A5A88F', 'sha256': 'BAF97B2A629723947539CFF84E896CD29565AB4BB68B0CEC515EB5C5D6637B69', 'imphash': 'F8DD0EF565DE87D97ABF9C62EA63EC21'}",C:\Windows\System32\conhost.exe,NaN,NaN,NaN,NaN,NaN,NaN,0.001013,0.000000,11,0.000000,3_Descendant,True
207,207,Process,SysMonProc,conhost.exe,#FF0000,6121378660455115461,0.002332,4,101,{47ab858c-e188-5eac-af03-000000000400},SCRANTON.dmevals.local,3152.0,DMEVALS\pbeesly,conhost.exe,C:\Windows\System32,\\?\C:\windows\system32\conhost.exe --headless --width 80 --height 25 --signal 0x474 --server 0x50c,"{'sha1': '11996F32DD85863A8C3BFF6D520F788A9211C8F7', 'md5': 'C5E9B1D1103EDCEA2E408E9497A5A88F', 'sha256': 'BAF97B2A629723947539CFF84E896CD29565AB4BB68B0CEC515EB5C5D6637B69', 'imphash': 'F8DD0EF565DE87D97ABF9C62EA63EC21'}",C:\Windows\System32\conhost.exe,NaN,NaN,NaN,NaN,NaN,NaN,0.001013,0.000000,11,0.000000,3_Descendant,True
213,213,Process,SysMonProc,conhost.exe,#FF0000,-5468381441766902673,0.000389,4,114,{47ab858c-e394-5eac-e303-000000000400},SCRANTON.dmevals.local,6508.0,DMEVALS\pbeesly,conhost.exe,C:\Windows\System32,\??\C:\windows\system32\conhost.exe 0xffffffff -ForceV1,"{'sha1': '11996F32DD85863A8C3BFF6D520F788A9211C8F7', 'md5': 'C5E9B1D1103EDCEA2E408E9497A5A88F', 'sha256': 'BAF97B2A629723947539CFF84E896CD29565AB4BB68B0CEC515EB5C5D6637B69', 'imphash': 'F8DD0EF565DE87D97ABF9C62EA63EC21'}",C:\Windows\System32\conhost.exe,NaN,NaN,NaN,NaN,NaN,NaN,0.001013,0.000000,11,0.000000,3_Descendant,True
219,219,Process,SysMonProc,conhost.exe,#FF0000,8589652588512368721,0.001899,5,86,{47ab858c-e1e4-5eac-b903-000000000400},SCRANTON.dmevals.local,4196.0,DMEVALS\pbeesly,conhost.exe,C:\Windows\System32,\??\C:\windows\system32\conhost.exe 0xffffffff -ForceV1,"{'sha1': '11996F32DD85863A8C3BFF6D520F788A9211C8F7', 'md5': 'C5E9B1D1103EDCEA2E408E9497A5A88F', 'sha256': 'BAF97B2A629723947539CFF84E896CD29565AB4BB68B0CEC515EB5C5D6637B69', 'imphash': 'F8DD0EF565DE87D97ABF9C62EA63EC21'}",C:\Windows\System32\conhost.exe,NaN,NaN,NaN,NaN,NaN,NaN,0.001013,0.000000,11,0.000000,3_Descendant,True
222,222,Process,SysMonProc,conhost.exe,#FF0000,4323078242600618180,0.065810,4,107,{47ab858c-e23d-5eac-c703-000000000400},SCRANTON.dmevals.local,3796.0,DMEVALS\pbeesly,conhost.exe,C:\Windows\System32,\??\C:\windows\system32\conhost.exe 0xffffffff -ForceV1,"{'sha1': '11996F32DD85863A8C3BFF6D520F788A9211C8F7', 'md5': 'C5E9B1D1103EDCEA2E408E9497A5A88F', 'sha256': 'BAF97B2A629723947539CFF84E896CD29565AB4BB68B0CEC515EB5C5D6637B69', 'imphash': 'F8DD0EF565DE87D97ABF9C62EA63EC21'}",C:\Windows\System32\conhost.exe,NaN,NaN,NaN,NaN,NaN,NaN,0.001013,0.000000,11,0.000000,3_Descendant,True
224,224,Process,SysMonProc,conhost.exe,#FF0000,-9046114462966284117,0.002332,4,101,{47ab858c-e144-5eac-aa03-000000000400},SCRANTON.dmevals.local,5156.0,DMEVALS\pbeesly,conhost.exe,C:\Windows\System32,\\?\C:\windows\system32\conhost.exe --headless --width 80 --height 25 --signal 0x54c --server 0x540,"{'sha1': '11996F32DD85863A8C3BFF6D520F788A9211C8F7', 'md5': 'C5E9B1D1103EDCEA2E408E9497A5A88F', 'sha256': 'BAF97B2A629723947539CFF84E896CD29565AB4BB68B0CEC515EB5C5D6637B69', 'imphash': 'F8DD0EF565DE87D97ABF9C62EA63EC21'}",C:\Windows\System32\conhost.exe,NaN,NaN,NaN,NaN,NaN,NaN,0.001013,0.000000,11,0.000000,3_Descendant,True
226,226,Process,SysMonProc,conhost.exe,#FF0000,6962781418096119833,0.000304,4,118,{47a

In [99]:
# PSEXEC - '{47ab858c-e558-5eac-f503-000000000400}'
M2, malicious_root_ancestor, ancestors, descendants, file_nodes, proc_nodes, ip_nodes = manual_recurse_pipeline(G, '{47ab858c-e558-5eac-f503-000000000400}')

Node Data:
{'properties': {'process_guid': '{47ab858c-e13c-5eac-a903-000000000400}', 'host': 'SCRANTON.dmevals.local', 'process_id': 8524, 'user': nan, 'process_image': 'â€®cod.3aka3.scr', 'process_image_path': 'C:\\ProgramData\\victim', 'command_line': '"C:\\ProgramData\\victim\\â€®cod.3aka3.scr" /S', 'hashes': {'sha1': '4B7FA56A4E85F88B98D11A6E018698AE3FBA5E62', 'md5': '9D1C5EF38E6073661C74660B3A71A76E', 'sha256': '0DF38A55D940F498478EB03683C94D4584236E100125B526A67650BA54DF4AE4', 'imphash': 'F00447512A354E59D39D2818AABA4A17'}, 'process_path': 'C:\\ProgramData\\victim\\â€®cod.3aka3.scr'}, '_node_type': 'Process', '_node_class': 'SysMonProc', '_display': 'â€®cod.3aka3.scr', '_color': '#FF0000', 'Orig_Node_ID': 3685170341351964961}


Malicious? Enter yes or no:  no



##########################################
Confirm this as the senior malicious node?
##########################################
Node Data:
{'properties': {'process_guid': '{47ab858c-e558-5eac-f503-000000000400}', 'host': 'SCRANTON.dmevals.local', 'process_id': 1412, 'user': 'DMEVALS\\pbeesly', 'process_image': 'PsExec64.exe', 'process_image_path': 'C:\\Program Files\\SysinternalsSuite', 'command_line': '"C:\\Program Files\\SysinternalsSuite\\PsExec64.exe" -accepteula \\\\NASHUA -u dmevals\\pbeesly -p Fl0nk3rt0n!T0by -i 2 C:\\Windows\\Temp\\python.exe', 'hashes': {'sha1': 'FB0A150601470195C47B4E8D87FCB3F50292BEB2', 'md5': '9321C107D1F7E336CDA550A2BF049108', 'sha256': 'AD6B98C01EE849874E4B4502C3D7853196F6044240D3271E4AB3FC6E3C08E9A4', 'imphash': '159D56D406180A332FBC99290F30700E'}, 'process_path': 'C:\\Program Files\\SysinternalsSuite\\PsExec64.exe'}, '_node_type': 'Process', '_node_class': 'SysMonProc', '_display': 'PsExec64.exe', '_color': '#FF0000', 'Orig_Node_ID': -9340272470070526

Malicious? Enter yes or no:  yes


838
838
1
17
19


In [100]:
malicious_root_ancestor

838

In [101]:
ancestors

{278,
 280,
 281,
 992,
 993,
 1017,
 1073,
 1076,
 1077,
 1365,
 1367,
 1409,
 1460,
 1461,
 1907,
 1998,
 2028}

In [103]:
df['Chain_Loc'] = np.nan
df['Chain_Loc'].loc[df.NodeID.isin(list(ancestors))] = '1_Ancestor' #[['_display', 'DegreeCentrality']]
df['Chain_Loc'].loc[df.NodeID.isin([malicious_root_ancestor])] = '2_Root' #[['_display', 'DegreeCentrality']]
df['Chain_Loc'].loc[df.NodeID.isin(list(descendants))] = '3_Descendant' #[['_display', 'DegreeCentrality']]
df.loc[((df['Chain_Loc']=='1_Ancestor') | (df['Chain_Loc']=='2_Root') | ((df['Chain_Loc']=='3_Descendant') & (df['_node_type']=='Process')))].sort_values(by=['Chain_Loc'], ascending=True)[['_display', '_node_type', 'Chain_Loc', 'BestPartition', 'Greedy_Mod', 'label_prop', 'betweenness', 'DegreeCentrality', 'ClusterCoef', 'AveNeighDegree']]

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,_display,_node_type,Chain_Loc,BestPartition,Greedy_Mod,label_prop,betweenness,DegreeCentrality,ClusterCoef,AveNeighDegree
278,powershell.exe,File,1_Ancestor,11,4,111,0.078199,0.006079,0.037879,42.250000
1870,powershell.exe,File,1_Ancestor,8,5,107,0.044976,0.001013,0.000000,226.000000
1434,Explorer.EXE,Process,1_Ancestor,13,9,146,0.009953,0.005572,0.000000,0.666667
1433,explorer.exe,File,1_Ancestor,13,9,146,0.000161,0.001013,0.000000,7.500000
1382,control.exe,File,1_Ancestor,11,5,7,0.000000,0.000507,0.000000,1.000000
1344,â€®cod.3aka3.scr,File,1_Ancestor,11,4,101,0.000000,0.000507,0.000000,6.000000
1342,â€®cod.3aka3.scr,Process,1_Ancestor,11,4,101,0.013714,0.004053,0.000000,1.000000
1062,sdclt.exe,File,1_Ancestor,11,4,8,0.000006,0.001520,0.000000,0.333333
1061,control.exe,Process,1_Ancestor,11,5,7,0.002369,0.001520,0.000000,19.000000
1058,sdclt.exe,Process,1_Ancestor,11,4,8,0.000823,0.001520,0.000000,1.000000


In [108]:
df['Chain_Loc_Int'] = np.nan
df['Chain_Loc_Int'].loc[df.NodeID.isin([malicious_root_ancestor])] = 0 #[['_display', 'DegreeCentrality']]
counter = 1
for i in ancestors:
    df['Chain_Loc_Int'].loc[df['NodeID'] == i] = counter #[['_display', 'DegreeCentrality']]
    counter += 1
df.loc[((df['Chain_Loc']=='1_Ancestor') | (df['Chain_Loc']=='2_Root') | ((df['Chain_Loc']=='3_Descendant') & (df['_node_type']=='Process')))].sort_values(by=['Chain_Loc_Int'], ascending=True)[['_display', '_node_type', 'Chain_Loc', 'Chain_Loc_Int', 'BestPartition', 'Greedy_Mod', 'label_prop', 'betweenness', 'DegreeCentrality', 'ClusterCoef', 'AveNeighDegree']]

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,_display,_node_type,Chain_Loc,Chain_Loc_Int,BestPartition,Greedy_Mod,label_prop,betweenness,DegreeCentrality,ClusterCoef,AveNeighDegree
825,PsExec64.exe,Process,2_Root,0.0,8,5,100,0.000253,0.001520,0.166667,0.000000
1382,control.exe,File,1_Ancestor,1.0,11,5,7,0.000000,0.000507,0.000000,1.000000
1924,PowerShell.exe,File,1_Ancestor,2.0,11,5,86,0.000000,0.000507,0.000000,19.000000
1342,â€®cod.3aka3.scr,Process,1_Ancestor,3.0,11,4,101,0.013714,0.004053,0.000000,1.000000
278,powershell.exe,File,1_Ancestor,4.0,11,4,111,0.078199,0.006079,0.037879,42.250000
1344,â€®cod.3aka3.scr,File,1_Ancestor,5.0,11,4,101,0.000000,0.000507,0.000000,6.000000
280,powershell.exe,Process,1_Ancestor,6.0,8,5,85,0.322120,0.065856,0.000621,0.125984
281,PowerShell.exe,Process,1_Ancestor,7.0,11,5,86,0.075196,0.011145,0.007143,24.000000
979,cmd.exe,File,1_Ancestor,8.0,11,4,174,0.008598,0.004053,0.017857,2.375000
980,cmd.exe,Process,1_Ancestor,9.0,11,4,8,0.005036,0.003546,0.000000,1.400000


## Start at the bottom test - Nashua